In [1]:
"""Modular aircraft concept"""
import pickle
import numpy as np
from gpkit import Model, Variable, Vectorize
from gpkit.constraints.tight import Tight
from gpkit.constraints.loose import Loose

from gpkit import ureg as u

from aircraft import *

from prettytable import PrettyTable

In [2]:
class SimpleAircraft(Model):
    
    def setup(self):
        
        constraints = []
        components  = []
        
        W_0       = self.W_0       = Variable('W_0', "N", "Weight (force) of aircraft")
        W_payload = self.W_payload = Variable('W_{payload}', "N", "Weight (force) of payload + crew")
        W_empty   = self.W_empty   = Variable('W_{empty}', "N", "Weight (force) of empty aircraft")
        W_fuel    = self.W_fuel    = Variable('W_{fuel}', "N", "Weight (force) of fuel")
        W_dry     = self.W_dry     = Variable('W_{dry}', "N", "Weight (force) of dry aircraft (payload + empty)")
        
        M_0       = self.M_0       = Variable('M_0', "kg", "Mass of aircraft")
        M_payload = self.M_payload = Variable('M_{payload}', "kg", "Mass of payload + crew")
        M_empty   = self.M_empty   = Variable('M_{empty}', "kg", "Mass of empty aircraft")
        M_fuel    = self.M_fuel    = Variable("M_{fuel}", "kg", "Mass of fuel")
        M_dry     = self.M_dry     = Variable("M_{dry}", "kg", "Mass of dry aircraft (payload + empty)")
        
        g_0 = self.g_0 = Variable("g_0", 9.81, "m/s^2", "Acceleration due to gravity")

        constraints += [Tight([W_0 >= W_dry + W_fuel])]
        constraints += [Tight([W_dry >= W_payload + W_empty])]
        
        constraints += [Tight([W_0 == M_0*g_0])]
        constraints += [Tight([W_dry == M_dry*g_0])]
        constraints += [Tight([W_fuel == M_fuel*g_0])]
        constraints += [Tight([W_payload == M_payload*g_0])]
        constraints += [Tight([W_empty == M_empty*g_0])]
        
        #payload mass estimate
        N_crew = self.N_crew = Variable("N_{crew}", 4., "", "number of crew")
        N_passengers = self.N_passengers = Variable("N_{passengers}", 50., "", "number of passengers")
        M_per_person = Variable("M_{per passenger}", 100., "kg", "mass per passenger") # todo: get real number, currently arbitrary
        
        constraints += [M_payload >= M_per_person*(N_crew + N_passengers)] # todo: refine this mass estimate
        #constraints += [M_payload >= 9500*u.kg] # todo: refine this mass estimate
        #todo this 9500 comes from CRJ500 -> using 54*100 is far smaller!!

        
        # empty weight fraction
        fit_A =  0.97 #Using Table 3.1 of Raymer, Jet transport
        fit_C = -0.06 #Using Table 3.1 of Raymer, Jet transport
        fit_K_vs = 1.00 # assumes fixed sweep
        constraints += [Tight([M_empty/M_0 >= fit_A*(M_0/(1.*u.kg))**fit_C*fit_K_vs])]
        
        
        
        # range equation
        R_total = self.R_total = Variable("R_{total}", 2500., "km", "Total range of aircraft")
        V_cruise = self.V_cruise = Variable("V_{cruise}", "m/s", "Cruise speed of aircraft")
        Ma_cruise = self.Ma_cruise = Variable("Ma_{cruise}", 0.75, "", "Cruise mach number")
        h_cruise  = self.h_cruise = Variable("h_{cruise}", 35000., "ft", "Cruise altitude")
        a_cruise = Variable("a_{cruise}", 295.2, "m/s", "Speed of sound at 36,089 ft") #https://www.engineeringtoolbox.com/elevation-speed-sound-air-d_1534.html
        
        LD_max = Variable("LD_{max}", "", "L/D max") #todo: eyeballed from Raymer Fig 3.5. Need to incorporate better approx
        
        S_wet_S_ref = Variable("S_{wet}/S_{ref}", 6., "", "S wet to S ref ratio") # eyeballed from Raymer Fig. 3.6
        AR = Variable("AR", 8., "", "Aspect ratio, main") #todo: arbitrary
        
        K_LD = Variable("K_LD", 15.5, "", "Coefficient for estimating max lift to drag")
        constraints += [Tight([LD_max <= K_LD*(AR/(S_wet_S_ref))])]
        
        SFC_cruise = self.SFC_cruise = Variable("SFC_{cruise}", 19.8, "mg/N/s", "Specific Fuel Consumption, Cruise") #Table 3.3 of Raymer, for Low bypass Turbofan
        
        z = Variable("z", "", "Dummy variable for range eqn")
        constraints += [z >= R_total*SFC_cruise*g_0/(V_cruise*LD_max)]
        constraints += [W_fuel/(W_dry) >= z + z**2/2 + z**3/6 + z**4/24]
        
        
        
        
        constraints += [Tight([V_cruise <= Ma_cruise*a_cruise])]
        
        
        return constraints, components
            
        
        

In [3]:
#create the simple aircraft
AC = SimpleAircraft()

# define the optimizer to the AC.W_0, and set constraints to be the AC
M = Model(AC.W_0, AC)

#print latex version of the constraints
M

<gpkit.Model object containing 1 top-level constraint(s) and 25 variable(s)>

In [13]:
# run a solve
sol = M.solve()

Using solver 'cvxopt'
Solving for 13 variables.
Solving took 0.0647 seconds.


In [5]:
# print(sol.summary()) 
#the summary which usually works well is a bit buggy here. Ive raised an appropriate github issue, and hopefully they will push a new version of gpkit that fixes it. 
#In the mean time ive written a rough version of it

In [6]:
x = PrettyTable()

x.field_names = ['Type', 'Var', 'Val', 'Unit', "Description"]

for k, v in sol['freevariables'].items():
    
    x.add_row(['Free', k, v, k.unitstr(), k.descr['label']])

    
for k, v in sol['constants'].items():
    
    x.add_row(['Fix', k, v, k.unitstr(), k.descr['label']])

x.float_format = '10.2'

print(x)

+------+----------------------------------+------------+--------+--------------------------------------------------+
| Type |               Var                |    Val     |  Unit  |                   Description                    |
+------+----------------------------------+------------+--------+--------------------------------------------------+
| Free |        SimpleAircraft.W_0        |  149220.19 |   N    |            Weight (force) of aircraft            |
| Free |      SimpleAircraft.W_{dry}      |  134195.27 |   N    | Weight (force) of dry aircraft (payload + empty) |
| Free |     SimpleAircraft.W_{fuel}      |   15024.91 |   N    |              Weight (force) of fuel              |
| Free |    SimpleAircraft.W_{payload}    |   52974.00 |   N    |         Weight (force) of payload + crew         |
| Free |     SimpleAircraft.W_{empty}     |   81221.27 |   N    |         Weight (force) of empty aircraft         |
| Free |        SimpleAircraft.M_0        |   15211.03 |   kg   

In [7]:
#full solution dictionary
sol

{'freevariables': {SimpleAircraft.W_0: 149220.18557802433,
  SimpleAircraft.W_{dry}: 134195.27370369475,
  SimpleAircraft.W_{fuel}: 15024.91302778077,
  SimpleAircraft.W_{payload}: 52973.999997423736,
  SimpleAircraft.W_{empty}: 81221.27373248017,
  SimpleAircraft.M_0: 15211.028091782666,
  SimpleAircraft.M_{dry}: 13679.436667318856,
  SimpleAircraft.M_{fuel}: 1531.5915421084014,
  SimpleAircraft.M_{payload}: 5399.999999846133,
  SimpleAircraft.M_{empty}: 8279.436670181301,
  SimpleAircraft.LD_{max}: 20.666666193497136,
  SimpleAircraft.V_{cruise}: 221.39999492634877,
  SimpleAircraft.z: 0.10612707258593806},
 'cost': 149220.18557802433 <Unit('newton')>,
 'constants': {SimpleAircraft.M_{per passenger}: 100.0,
  SimpleAircraft.N_{passengers}: 50.0,
  SimpleAircraft.a_{cruise}: 295.2,
  SimpleAircraft.g_0: 9.81,
  SimpleAircraft.SFC_{cruise}: 19.8,
  SimpleAircraft.h_{cruise}: 35000.0,
  SimpleAircraft.Ma_{cruise}: 0.75,
  SimpleAircraft.S_{wet}/S_{ref}: 6.0,
  SimpleAircraft.R_{total}: 

In [8]:
# this is for the more complicated airplane, so ignore it.

#AC = Aircraft()
#MISSION = Mission(AC)
#M = Model(MISSION.takeoff_fuel, [MISSION, AC])

In [9]:
#AC

In [10]:
#print(M)

In [11]:
#sol = M.solve(verbosity=0)
#print(sol.summary())